<a href="https://colab.research.google.com/github/touriachi/Lunch_Learn_01/blob/main/Optimize_01" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:



!python --version

Python 3.8.10


In [5]:
!pip install numpy==1.14.0
import numpy as np
print(np.__version__)

!pip install pandas==1.4.2
import pandas as pd
print(pd.__version__)


!pip install scipy==1.7.0
import scipy as sc
print(sc.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 25.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
  Running setup.py clean for numpy
  error: subprocess-exited-with-error
  
  × python setup.py clean did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed cleaning build dir for numpy
Failed to build numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  error: subprocess-exite

In [6]:


from scipy.optimize import minimize

In [14]:
def optimize_portfolio(covariance_matrix, industry_classification, company_identifiers):
    n = covariance_matrix.shape[0]

    def objective(x):
        return np.sqrt(np.dot(x.T, np.dot(covariance_matrix, x)))

    def constraint1(x):
           if  (np.sum(x) - 1.0) >= 0 :
               return 1
           else :  
               return 0
        
    def constraint2(x):
        industries = industry_classification['Industry'].unique()
        for industry in industries:
            mask = industry_classification['Industry'] == industry
            if np.sum(x[mask]) > 0.25:
                return 0
        return 1

    def constraint3(x):

         if  np.all(x >= 0) and np.all(x <= 0.08) :
            return  1
         else  :
            return 0   
        

    constraints = [{'type': 'eq', 'fun': constraint1},
                   {'type': 'ineq', 'fun': constraint2},
                   {'type': 'ineq', 'fun': constraint3}]

    x0 = np.ones(n) / n
    result = minimize(objective, x0, constraints=constraints)

    optimal_weights = result.x

    df = pd.DataFrame({'Industry': industry_classification['Industry'],
                       'Optimal Weight': optimal_weights}, index=company_identifiers)
    return df


In [15]:
# Example data for the covariance matrix
covariance_matrix = pd.DataFrame({'Stock 1': [0.03, 0.02, 0.01],
                                   'Stock 2': [0.02, 0.05, 0.03],
                                   'Stock 3': [0.01, 0.03, 0.04]},
                                  index=['Stock 1', 'Stock 2', 'Stock 3'])

# Example data for the industry classification
industry_classification = pd.DataFrame({'Industry': ['Technology', 'Finance', 'Finance']},
                                        index=['Stock 1', 'Stock 2', 'Stock 3'])

# Example list of company identifiers
company_identifiers = ['Stock 1', 'Stock 2', 'Stock 3']

# Call the optimize_portfolio function
result = optimize_portfolio(covariance_matrix, industry_classification, company_identifiers)
print(result)

           Industry  Optimal Weight
Stock 1  Technology        0.333333
Stock 2     Finance        0.333333
Stock 3     Finance        0.333333
